In [41]:
import json
import re
from typing import Dict, List, Tuple, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [ ]:
def most_recent_file():
    import os
    import glob
    list_of_files = glob.glob('logs/*') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    return latest_file


def load_log_file(file_path: str):
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Split the file into three sections
    sections = re.split(r'\n\n+(?:Sandbox logs:|Activities log:|Trade History:)\n', content)
    
    # Parse each section
    assert len(sections) == 3, "Expected 3 sections in the log file"


    sandbox_logs = parse_sandbox_logs(sections[0])
    activities_log = parse_activities_log(sections[1])
    trade_history = parse_trade_history(sections[2])
    
    return sandbox_logs, activities_log, trade_history


def parse_sandbox_logs(logs_text: str):
    """Parse the sandbox logs section into a list of log entries."""
    # Split the logs into individual JSON entries
    print('[' + logs_text.replace('\n', ',') + ']')

    log_entries = json.loads('[' + logs_text.replace('\n', ',') + ']')
    return pd.DataFrame(log_entries)




def parse_activities_log(activities_text: str) -> pd.DataFrame:
    """Parse the activities log section into a pandas DataFrame."""
    # Split the text into lines
    lines = activities_text.strip().split('\n')
    
    # Extract the header
    header = lines[0].split(';')
    
    # Parse the data rows
    data = []
    for line in lines[1:]:
        if line.strip():  # Skip empty lines
            values = line.split(';')
            data.append(values)
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=header)
    
    # Convert numeric columns
    numeric_columns = [
        'timestamp', 
        'bid_price_1', 'bid_volume_1', 
        'bid_price_2', 'bid_volume_2',
        'bid_price_3', 'bid_volume_3',
        'ask_price_1', 'ask_volume_1',
        'ask_price_2', 'ask_volume_2',
        'ask_price_3', 'ask_volume_3',
        'mid_price', 'profit_and_loss'
    ]
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df


def parse_trade_history(trade_text: str) -> pd.DataFrame:
    """Parse the trade history section into a pandas DataFrame."""
    
    trades = json.loads(trade_text)
    df = pd.DataFrame(trades)
    return df


    

In [54]:
file_path = most_recent_file()
sandbox_logs, activities_log, trade_history = load_log_file(file_path)

logs\91a2c45e-5b0a-4a55-9a90-f14e7dda8d3b.log
[Sandbox logs:,{,  "sandboxLog": "",,  "lambdaLog": "traderData: \nObservations: (plainValueObservations: {}, conversionObservations: {})\nAcceptable price : 10\nBuy Order depth : 3, Sell order depth : 1\nSELL 1x 2028\nAcceptable price : 10\nBuy Order depth : 3, Sell order depth : 2\nSELL 1x 10002",,  "timestamp": 0,},{,  "sandboxLog": "",,  "lambdaLog": "traderData: SAMPLE\nObservations: (plainValueObservations: {}, conversionObservations: {})\nAcceptable price : 10\nBuy Order depth : 1, Sell order depth : 2\nSELL 24x 2025\nAcceptable price : 10\nBuy Order depth : 2, Sell order depth : 2\nSELL 2x 9996",,  "timestamp": 100,},{,  "sandboxLog": "",,  "lambdaLog": "traderData: SAMPLE\nObservations: (plainValueObservations: {}, conversionObservations: {})\nAcceptable price : 10\nBuy Order depth : 1, Sell order depth : 1\nSELL 22x 2025\nAcceptable price : 10\nBuy Order depth : 1, Sell order depth : 1\nSELL 20x 9995",,  "timestamp": 200,},{,  "sa

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [39]:
activities_log

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-1,0,RAINFOREST_RESIN,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,0.000000
1,-1,0,KELP,2028,1,2026.0,2.0,2025.0,29.0,2029,31,NaN,NaN,NaN,NaN,2028.5,0.000000
2,-1,100,KELP,2025,24,NaN,NaN,NaN,NaN,2028,2,2029.0,22.0,NaN,NaN,2026.5,1.157471
3,-1,100,RAINFOREST_RESIN,9996,2,9995.0,22.0,NaN,NaN,10004,2,10005.0,22.0,NaN,NaN,10000.0,2.000000
4,-1,200,RAINFOREST_RESIN,9995,20,NaN,NaN,NaN,NaN,10005,20,NaN,NaN,NaN,NaN,10000.0,-6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-1,199700,KELP,2020,27,NaN,NaN,NaN,NaN,2023,2,2024.0,25.0,NaN,NaN,2021.5,162.531250
3996,-1,199800,RAINFOREST_RESIN,9995,21,NaN,NaN,NaN,NaN,10002,8,10005.0,21.0,NaN,NaN,9998.5,-180.000000
3997,-1,199800,KELP,2020,21,NaN,NaN,NaN,NaN,2023,21,NaN,NaN,NaN,NaN,2021.5,164.593750
3998,-1,199900,KELP,2020,32,NaN,NaN,NaN,NaN,2022,7,2023.0,2.0,2024.0,30.0,2021.0,160.351562


In [40]:
trade_history

,timestamp,buyer,seller,symbol,currency,price,quantity
0,0,,SUBMISSION,KELP,SEASHELLS,2028,1
1,0,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,1
2,0,,,KELP,SEASHELLS,2029,13
3,0,,,KELP,SEASHELLS,2029,1
4,0,,,RAINFOREST_RESIN,SEASHELLS,10004,1
...,...,...,...,...,...,...,...
1107,199400,,,RAINFOREST_RESIN,SEASHELLS,10004,1
1108,199500,,,RAINFOREST_RESIN,SEASHELLS,10002,1
1109,199600,,,KELP,SEASHELLS,2023,1
1110,199600,,,KELP,SEASHELLS,2023,1
